# Player evaluation

Here we compare players based on their adjusted points. Also, we calculate some useful metrics using the adjusted points to further evaluate player performance in FPL.

In [1]:
# recent gameweek
gameweek = 25

In [2]:
# import basic libraries
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [3]:
# fetch FPL data
filepath = '../data/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)#, encoding='latin-1')

# fetch team data
filepath = '../data/fbref/team_stats_week' + str(gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0)#, encoding='latin-1')

# fetch player stats data
filepath = '../data/fbref/player_stats_week' + str(gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)#, encoding='latin-1')

## Value and value points

Here we calculate two interesting metrics: 'value' and 'value points'. 

Value is simply adjusted points per game divided by the cost of the player. Essentially, this measures how many 'points per pound(/euro/whatever)' has the player gained on an average game week. 

The value points is calculated as the geometric mean of a players adjusted points per game and value. Essentially, in FPL we want to gain the maximum possible amount of points. However, we are restrained by a limited budget, so that in practice we cannot just pick all the best players. Then, we would like to have a squad of players that give the most points given our budget. Thus, we would like to pick players that have high value ('bang for buck'). However, often the players with highest value are very cheap players who somewhat overperform relative to their price. Picking a squad full of these players might leave a part of our budget unused. Calculating the geometric mean of adjusted points and value allows us to weigh both aspects equally, i.e. we want players that both gain a lot of points but also have good value. For example, this metric values equally two players, where one has twice the points per game of the other who in turn has twice the value of the other.

In [4]:
# value = expected points / cost
df['value'] = df['adjusted points per game'] / (df['now_cost'] / 10.0)
# geometric mean of 'adjusted points per game' and 'value'
df['valuePoints metric'] = np.sqrt(df['adjusted points per game'] * df['value'])

In [5]:
# assign proper team names for each player
team_names = np.sort(teamStats['Squad'])
df['team_name'] = team_names[df['team']-1]

In [6]:
# save data
filepath = '../data/data_week' + str(gameweek) + str('.csv')
df.to_csv(filepath)

In [7]:
goalkeepers = df['element_type'] == 1
defenders = df['element_type'] == 2
midfielders = df['element_type'] == 3
forwards = df['element_type'] == 4

minGames = df['games played'] >= 3

## Player evaluation

Below we compile lists for each position sorting players based on a given metric.

In [8]:
metric = 'valuePoints metric'
#metric = 'adjusted points per game'
#metric = 'form 5'

numberToShow = 40

print('GOALKEEPERS')
display(df[goalkeepers & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('DEFENDERS')
display(df[defenders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('MIDFIELDERS')
display(df[midfielders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('FORWARDS')
display(df[forwards & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

GOALKEEPERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
340,Lloris,Tottenham,53,9.090909,40,41.221791,4.4,4.534397,7.680326,7.433607,1.969616
341,Gazzaniga,Tottenham,44,16.764706,57,67.206068,3.4,4.008783,6.954244,4.910144,1.911112
411,Patrício,Wolves,51,25.277778,91,108.048209,3.6,4.274435,4.359166,4.216694,1.892751
471,Henderson,Sheffield Utd,51,24.130435,111,101.657565,4.6,4.212836,6.572768,5.608263,1.865475
93,Pope,Burnley,46,25.000000,105,99.165403,4.2,3.966616,5.525301,4.862991,1.849444
494,Ramsdale,Bournemouth,46,24.166667,87,90.099213,3.6,3.728243,4.036462,3.904054,1.738302
47,Ryan,Brighton,49,24.722222,89,92.807410,3.6,3.754008,2.907138,3.293550,1.695888
131,Guaita,Crystal Palace,51,22.894737,87,87.366309,3.8,3.816000,3.015449,3.713901,1.689753
14,Leno,Arsenal,50,24.722222,89,92.335992,3.6,3.734939,3.387055,3.689154,1.670315


DEFENDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
297,Lundstram,Sheffield Utd,51,24.000000,108,108.704708,4.5,4.529363,2.266692,3.043871,2.005636
182,Alexander-Arnold,Liverpool,76,25.172414,146,136.761051,5.8,5.432973,4.383933,4.937274,1.970747
185,Matip,Liverpool,52,6.956522,32,29.601749,4.6,4.255251,NaN,NaN,1.866051
208,Otamendi,Manchester City,50,15.833333,57,65.442832,3.6,4.133231,4.549608,4.089510,1.848437
181,Robertson,Liverpool,70,25.102041,123,122.044844,4.9,4.861949,4.118455,4.383220,1.837644
105,Azpilicueta,Chelsea,58,22.941176,78,100.922431,3.4,4.399183,5.456509,5.891144,1.826661
405,Boly,Wolves,47,9.090909,40,35.925862,4.4,3.951845,3.540000,3.540000,1.822850
103,Alonso,Chelsea,60,7.966102,47,34.960613,5.9,4.388673,NaN,6.851449,1.791668
401,Doherty,Wolves,61,23.055556,83,100.496715,3.6,4.358894,3.920144,4.081093,1.764864


MIDFIELDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
191,Salah,Liverpool,127,22.000000,165,156.694864,7.5,7.122494,8.602122,8.437651,1.998620
215,De Bruyne,Manchester City,107,24.029851,161,146.180371,6.7,6.083283,5.162874,5.249987,1.859714
192,Mané,Liverpool,122,21.940299,147,138.116208,6.7,6.295092,4.286282,5.255809,1.802280
431,Pulisic,Chelsea,69,15.909091,70,72.924308,4.4,4.583814,2.856183,2.722011,1.745028
417,Traoré,Wolves,58,23.953488,103,98.089630,4.3,4.095004,4.236143,4.357147,1.700358
287,Cantwell,Norwich City,49,24.864865,92,92.787032,3.7,3.731652,2.952291,3.385653,1.685789
239,Martial,Manchester Utd,79,18.823529,96,89.105901,5.1,4.733751,3.843724,4.101676,1.684193
301,Fleck,Sheffield Utd,51,21.750000,87,81.328483,4.0,3.739241,3.224526,3.819536,1.655764
463,Mount,Chelsea,61,24.750000,99,99.972711,4.0,4.039301,4.452829,3.810322,1.635465


FORWARDS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
278,Pukki,Norwich City,65,23.800000,119,120.345491,5.0,5.056533,6.630729,5.719754,1.983336
233,Rashford,Manchester Utd,89,21.967213,134,125.649151,6.1,5.719849,6.803924,5.461512,1.917298
409,Jiménez,Wolves,77,25.192308,131,127.970982,5.2,5.079764,5.026900,5.093849,1.830620
460,Abraham,Chelsea,77,24.074074,130,120.209151,5.4,4.993303,4.244096,3.874803,1.799462
166,Vardy,Leicester City,98,23.030303,152,129.690982,6.6,5.631319,2.902865,3.818143,1.798859
313,Ings,Southampton,70,25.098039,128,117.901830,5.1,4.697651,3.672261,4.566737,1.775545
362,Deeney,Watford,63,13.947368,53,58.461830,3.8,4.191603,4.913943,4.834598,1.669975
187,Firmino,Liverpool,97,24.897959,122,127.214642,4.9,5.109441,6.995824,6.225558,1.640542
91,Wood,Burnley,62,22.790698,98,92.901830,4.3,4.076305,5.352507,4.503644,1.637083


Below we compile a list sorting players based on a given metric (irrespective of position).

In [9]:
metric = 'adjusted points per game'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
191,Salah,Liverpool,127,22.000000,165,156.694864,7.5,7.122494,8.602122,8.437651,1.998620
192,Mané,Liverpool,122,21.940299,147,138.116208,6.7,6.295092,4.286282,5.255809,1.802280
215,De Bruyne,Manchester City,107,24.029851,161,146.180371,6.7,6.083283,5.162874,5.249987,1.859714
233,Rashford,Manchester Utd,89,21.967213,134,125.649151,6.1,5.719849,6.803924,5.461512,1.917298
166,Vardy,Leicester City,98,23.030303,152,129.690982,6.6,5.631319,2.902865,3.818143,1.798859
214,Sterling,Manchester City,117,23.200000,116,127.850048,5.0,5.510778,4.540621,4.483808,1.611091
182,Alexander-Arnold,Liverpool,76,25.172414,146,136.761051,5.8,5.432973,4.383933,4.937274,1.970747
210,Agüero,Manchester City,120,18.906250,121,100.547321,6.4,5.318205,5.988725,5.411916,1.535234
187,Firmino,Liverpool,97,24.897959,122,127.214642,4.9,5.109441,6.995824,6.225558,1.640542


In [10]:
metric = 'form 10'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
191,Salah,Liverpool,127,22.000000,165,156.694864,7.5,7.122494,8.602122,8.437651,1.998620
340,Lloris,Tottenham,53,9.090909,40,41.221791,4.4,4.534397,7.680326,7.433607,1.969616
103,Alonso,Chelsea,60,7.966102,47,34.960613,5.9,4.388673,NaN,6.851449,1.791668
187,Firmino,Liverpool,97,24.897959,122,127.214642,4.9,5.109441,6.995824,6.225558,1.640542
105,Azpilicueta,Chelsea,58,22.941176,78,100.922431,3.4,4.399183,5.456509,5.891144,1.826661
278,Pukki,Norwich City,65,23.800000,119,120.345491,5.0,5.056533,6.630729,5.719754,1.983336
471,Henderson,Sheffield Utd,51,24.130435,111,101.657565,4.6,4.212836,6.572768,5.608263,1.865475
265,Pérez,Leicester City,62,22.127660,104,89.409851,4.7,4.040637,5.431544,5.483342,1.622759
233,Rashford,Manchester Utd,89,21.967213,134,125.649151,6.1,5.719849,6.803924,5.461512,1.917298
